# Spaceship Titanic

In [ ]:
import pandas as pd
import plotly

: 

In [ ]:
train_csv = "../data/train.csv"
test_csv = "../data/test.csv"

: 

In [ ]:
train_df = pd.read_csv(train_csv)
train_df.head()

: 

In [ ]:
train_df.describe()

: 

In [ ]:
train_df.nunique(dropna=False)

: 

In [ ]:
train_df.Destination.value_counts()

: 

In [ ]:
for col in train_df.columns:
    if (train_df[col].nunique(dropna=False)) < 10:
        display(pd.DataFrame((train_df[col].value_counts(dropna=False))))

: 

### Initial ideas
* Most columns need to be cleaned of NaN values
    * Try dropping them, or replacing them with 'None'
    * Some models can interpret these values
    * But also want to try Predictive Imputing of missing values
* How does PassengerID work?
    * Would be interesting to look at how many groups there are; split this into an extra column so that you have GroupID and passengerID
* How does Cabin work? 
    * Would be interesting to see how the Deck, Number and Side (Port or Starboard) affected the results too.
* Should Age be made discrete?
* Should money spent on the RoomService, FoodCourt, Shoppingmall, Spa, VRDeck be made discrete? 
    * do some people spend nothing? 
    * should each column be put into a range, or the sum of the columns make a range?
* How many people were transported, as a percentage?


In [ ]:
## "How does PassengerId work?"
train_df["GroupId"] = train_df["PassengerId"].apply(lambda row: row.split("_")[0])
desired_column_order = ["Name", "GroupId", "PassengerId", "HomePlanet", "CryoSleep", "Cabin", "Destination", "Age", "VIP", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "Transported"]
train_df = train_df[desired_column_order]
train_df.nunique(dropna=False)

train_df.head(1)

: 

In [ ]:
## "How does Cabin work?"
extra_cabin_cols = train_df.Cabin.str.split("/", expand=True).rename(columns={0: "Deck", 1:"Number", 2:"Side"})

train_df = train_df.assign(
    Deck=extra_cabin_cols["Deck"],
    Number=extra_cabin_cols["Number"],
    Side=extra_cabin_cols["Side"],
)

desired_column_order = [
    "Name", "GroupId", "PassengerId", 
    "HomePlanet", "CryoSleep", "Cabin", 
    "Deck", "Number", "Side", "Destination", 
    "Age", "VIP", "RoomService", "FoodCourt", 
    "ShoppingMall", "Spa", "VRDeck", "Transported"
    ]

train_df = train_df[desired_column_order]
train_df.head(1)

: 

In [ ]:
## "How many people were Transported?"


: 

### Unrelated questions

* Is there a relationship between VIP status and $ spent on amenities? 
* Is there a relationship between location on the ship and Transported?

: 